In [103]:
pip install imbalanced-learn


#Bibliotecas

In [104]:
#Importes
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

#Carregando os Dados

In [105]:
# Carregando o conjunto de dados
url = "https://archive.ics.uci.edu/static/public/579/data.csv"
dados = pd.read_csv(url)


In [106]:
dados.head()

,ID,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,IBS_NASL,GB,SIM_GIPERT,...,JELUD_TAH,FIBR_JELUD,A_V_BLOK,OTEK_LANC,RAZRIV,DRESSLER,ZSN,REC_IM,P_IM_STEN,LET_IS
0,1,77.0,1,2.0,1.0,1.0,2.0,NaN,3.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2,55.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,3,52.0,1,0.0,0.0,0.0,2.0,NaN,2.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,4,68.0,0,0.0,0.0,0.0,2.0,NaN,2.0,0.0,...,0,0,0,0,0,0,1,0,0,0
4,5,60.0,1,0.0,0.0,0.0,2.0,NaN,3.0,0.0,...,0,0,0,0,0,0,0,0,0,0


#Tratando os dados

In [107]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1700 entries, 0 to 1699
Columns: 124 entries, ID to LET_IS
dtypes: float64(110), int64(14)
memory usage: 1.6 MB


In [108]:
dados.describe()

,ID,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,IBS_NASL,GB,SIM_GIPERT,...,JELUD_TAH,FIBR_JELUD,A_V_BLOK,OTEK_LANC,RAZRIV,DRESSLER,ZSN,REC_IM,P_IM_STEN,LET_IS
count,1700.000000,1692.000000,1700.000000,1696.000000,1594.000000,1627.000000,1649.000000,72.00000,1691.000000,1692.000000,...,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000
mean,850.500000,61.856974,0.626471,0.554835,2.316186,1.205286,1.160703,0.37500,1.393258,0.033688,...,0.024706,0.041765,0.033529,0.093529,0.031765,0.044118,0.231765,0.093529,0.087059,0.477059
std,490.892045,11.259936,0.483883,0.836801,2.440586,1.040814,0.801400,0.48752,1.088803,0.180478,...,0.155273,0.200110,0.180067,0.291259,0.175425,0.205417,0.422084,0.291259,0.282004,1.381818
min,1.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,425.750000,54.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,850.500000,63.000000,1.000000,0.000000,1.000000,2.000000,1.000000,0.00000,2.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1275.250000,70.000000,1.000000,1.000000,5.000000,2.000000,2.000000,1.00000,2.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1700.000000,92.000000,1.000000,3.000000,6.000000,4.000000,2.000000,1.00000,3.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000


In [109]:
#substituindo os nomes das colunas por numeros para facilitar a manipulação
nome_colunas = [str(i) for i in range(1, len(dados.columns) +1)]
dados = dados.rename(columns=dict(zip(dados.columns, nome_colunas)))
dados.head()

,1,2,3,4,5,6,7,8,9,10,...,115,116,117,118,119,120,121,122,123,124
0,1,77.0,1,2.0,1.0,1.0,2.0,NaN,3.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2,55.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,3,52.0,1,0.0,0.0,0.0,2.0,NaN,2.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,4,68.0,0,0.0,0.0,0.0,2.0,NaN,2.0,0.0,...,0,0,0,0,0,0,1,0,0,0
4,5,60.0,1,0.0,0.0,0.0,2.0,NaN,3.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [110]:
#Convertendo a variável LET_IS de categórico para binária
#[0] vivo [1] morto

def sub(valor):
    if valor > 0:
        return 1
    else:
        return valor

dados['124'] = dados['124'].apply(lambda x: sub(x))
dados['124'].value_counts()

124
0    1429
1     271
Name: count, dtype: int64

In [111]:
# Usar set em cada coluna para obter valores únicos
valores_unicos = {}

for coluna in dados.columns:
  col = dados[coluna].dropna()
  valores_unicos_sem_nulos = set(col)
  valores_unicos[coluna] = valores_unicos_sem_nulos

# Imprimir valores únicos de cada coluna
for coluna, conjunto in valores_unicos.items():
    print(f"Valores únicos em {coluna}: {conjunto}")

Valores únicos em 1: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 2

In [112]:
#Obtendo o número de valores nulos por coluna
#Adicionando colunas com mais de 30% de valores ausentes em uma lista
colunas_com_muitos_nulos = []
for coluna in dados.columns:
  nulos = dados[coluna].isnull().sum()
  if nulos > (1700*0.3):
    colunas_com_muitos_nulos.append(coluna)
  print(f'{coluna}: {nulos}')


1: 0
2: 8
3: 0
4: 4
5: 106
6: 73
7: 51
8: 1628
9: 9
10: 8
11: 248
12: 54
13: 21
14: 21
15: 21
16: 21
17: 21
18: 21
19: 21
20: 18
21: 18
22: 18
23: 18
24: 18
25: 18
26: 18
27: 11
28: 10
29: 10
30: 7
31: 7
32: 7
33: 7
34: 7
35: 1076
36: 1076
37: 267
38: 267
39: 12
40: 15
41: 14
42: 12
43: 12
44: 12
45: 83
46: 80
47: 80
48: 72
49: 1
50: 152
51: 152
52: 152
53: 152
54: 152
55: 152
56: 115
57: 115
58: 115
59: 115
60: 115
61: 115
62: 115
63: 115
64: 115
65: 115
66: 115
67: 115
68: 115
69: 115
70: 115
71: 115
72: 115
73: 115
74: 115
75: 115
76: 10
77: 10
78: 10
79: 10
80: 10
81: 10
82: 10
83: 369
84: 371
85: 375
86: 375
87: 284
88: 285
89: 1696
90: 125
91: 203
92: 126
93: 16
94: 108
95: 128
96: 657
97: 686
98: 677
99: 9
100: 5
101: 108
102: 131
103: 10
104: 110
105: 131
106: 10
107: 11
108: 13
109: 17
110: 17
111: 16
112: 16
113: 0
114: 0
115: 0
116: 0
117: 0
118: 0
119: 0
120: 0
121: 0
122: 0
123: 0
124: 0


In [113]:
colunas_com_muitos_nulos

['8', '35', '36', '89', '96', '97', '98']

In [114]:
#Excluindo colunas com mais de 30% de valores ausentes
dados = dados.drop(colunas_com_muitos_nulos,axis=1)

#Removendo a coluna 1(ID)
dados = dados.drop(['1'],axis=1)

In [115]:
#Substituindo os valores ausentes pela média dos valores em cada coluna
dados.fillna(dados.mean(), inplace=True)
#dados = dados.apply(lambda col: col.fillna(col.mode()[0]))

#Dividindo os dados de treino 60%, teste 20% e validação 20%

In [116]:
#Dividindo características e objetivo
#X = dados.iloc[:, :115]
X = dados.iloc[:, :104]
y = dados['124']
X.shape, y.shape

((1700, 104), (1700,))

##Normalização

In [117]:
#Normalização
norm = StandardScaler()

In [118]:
X_norm =norm.fit_transform(X)
X = pd.DataFrame(X_norm,columns=X.columns.tolist())

##Divisão dos dados e aplicando SMOTE

In [119]:
# Criar uma instância de SMOTE
smote = SMOTE(random_state=42)

# Aplicar SMOTE aos dados de treinamento
X_train_test_resampled, y_train_test_resampled = smote.fit_resample(X, y)

# Dados de treino e teste
X_train_test, X_val, y_train_test, y_val = train_test_split(X_train_test_resampled, y_train_test_resampled, test_size=0.2, random_state=42)


# Agora dividimos novamente X_train_val e y_train_val em treino e validação
X_train, X_test, y_train, y_test = train_test_split(X_train_test, y_train_test, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2

#Treinamento utilizando informações no momento da admissão no hospital

In [120]:
#Removendo colunas que não serão utilizadas no trienamento com informações no momento da admissão
X_test_1 = X_test.drop(['93', '94', '95', '100', '101', '102', '103', '104', '105'],axis=1)
X_train_1 = X_train.drop(['93', '94', '95', '100', '101', '102', '103', '104', '105'],axis=1)
X_val_1 = X_val.drop(['93', '94', '95', '100', '101', '102', '103', '104', '105'],axis=1)
X_admissao = X_train_test.drop(['93', '94', '95', '100', '101', '102', '103', '104', '105'],axis=1)

##Função para selecionar as melhores características para treinamento dos modelos

In [121]:
def select_features(model, X, y):
  rfe = RFE(estimator=model, n_features_to_select=40)
  fit = rfe.fit(X,y)
  cols = fit.get_support(indices=True)
  return cols

##Função para gerar relatórios

In [122]:
def relatorio(y_val, y_pred):
  # Gerar a matriz de confusão
  conf_matrix = confusion_matrix(y_val, y_pred)

  # Exibir a matriz de confusão
  print("Matriz de Confusão:")
  print(conf_matrix)
  print()
  # Exibir o relatório de classificação
  print("Relatório de Classificação:")
  print(classification_report(y_val, y_pred))

##Função para treinar diferentes modelos

In [123]:
def pred_models(model, X_train_test, y_train_test, X_test, X_train, y_test, y_train, X_val, y_val, results):
  #Selecionando features
  cols = select_features(model, X_train_test, y_train_test)
  X_test = X_test.iloc[:,cols]
  X_train = X_train.iloc[:,cols]
  X_val = X_val.iloc[:,cols]

  #Treinar o modelo
  model.fit(X_train, y_train)

  #Testar o modelo
  y_pred = model.predict(X_test)

  #Avaliar o desempenho
  accuracy = accuracy_score(y_test, y_pred)
  print(f"Acurácia: {accuracy}")


  print('###############################################################')
  #Avaliar o desempenho
  print('Validação do modelo')
  print('###############################################################')
  #validando o modelo
  y_pred = model.predict(X_val)
  accuracy = accuracy_score(y_val, y_pred)
  print()
  print(f"Acurácia: {accuracy}")
  results.append(accuracy)
  print()
  print('Features selecionadas')
  print(cols)
  print()
  relatorio(y_val, y_pred)


In [124]:
results_ad = []

##SVM

In [125]:
#modelo SVM
svm_model = SVC(kernel='linear')

pred_models(svm_model, X_admissao, y_train_test, X_test_1, X_train_1, y_test, y_train, X_val_1, y_val, results_ad)



Acurácia: 0.8356643356643356
###############################################################
Validação do modelo
###############################################################

Acurácia: 0.8094405594405595

Features selecionadas
[ 0  3  4  5  8  9 10 15 19 25 26 27 28 29 32 35 38 40 42 44 45 46 47 48
 49 50 52 58 59 60 62 63 70 82 84 86 87 88 90 92]

Matriz de Confusão:
[[224  64]
 [ 45 239]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.83      0.78      0.80       288
           1       0.79      0.84      0.81       284

    accuracy                           0.81       572
   macro avg       0.81      0.81      0.81       572
weighted avg       0.81      0.81      0.81       572



##KNN

In [126]:
#modelo KNN
model = KNeighborsClassifier(n_neighbors=5)

#Treinar o modelo
model.fit(X_train_1, y_train)

#Testar o modelo
y_pred = model.predict(X_test_1)

#Avaliar o desempenho
accuracy = accuracy_score(y_test, y_pred)
print(f"#Acurácia: {accuracy}")

print('###############################################################')
#Avaliar o desempenho
print('Validação do modelo')
print('###############################################################')
#validando o modelo
y_pred = model.predict(X_val_1)
accuracy = accuracy_score(y_val, y_pred)
print()
print(f"Acurácia: {accuracy}")
results_ad.append(accuracy)
print()
relatorio(y_val, y_pred)


#Acurácia: 0.8409090909090909
###############################################################
Validação do modelo
###############################################################

Acurácia: 0.798951048951049

Matriz de Confusão:
[[185 103]
 [ 12 272]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.94      0.64      0.76       288
           1       0.73      0.96      0.83       284

    accuracy                           0.80       572
   macro avg       0.83      0.80      0.79       572
weighted avg       0.83      0.80      0.79       572



##Regressão Logística

In [127]:
#modelo Regressão Logística
log_model = LogisticRegression(max_iter=1000)

pred_models(log_model, X_admissao, y_train_test, X_test_1, X_train_1, y_test, y_train, X_val_1, y_val, results_ad)


Acurácia: 0.8234265734265734
###############################################################
Validação do modelo
###############################################################

Acurácia: 0.8006993006993007

Features selecionadas
[ 0  3  4  5  8  9 10 11 15 16 19 25 26 27 28 32 35 38 40 42 44 46 48 49
 58 59 60 62 66 70 74 76 82 84 86 87 88 89 90 92]

Matriz de Confusão:
[[223  65]
 [ 49 235]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.82      0.77      0.80       288
           1       0.78      0.83      0.80       284

    accuracy                           0.80       572
   macro avg       0.80      0.80      0.80       572
weighted avg       0.80      0.80      0.80       572



##Random Forest

In [128]:
#modelo Random Forest
RF_model = RandomForestClassifier(n_estimators=100, random_state=42)

pred_models(RF_model, X_admissao, y_train_test, X_test_1, X_train_1, y_test, y_train, X_val_1, y_val, results_ad)


Acurácia: 0.9423076923076923
###############################################################
Validação do modelo
###############################################################

Acurácia: 0.958041958041958

Features selecionadas
[ 0  1  2  3  4  5  6  8  9 14 24 25 28 32 33 35 40 41 42 43 44 45 46 49
 70 78 79 80 81 82 83 84 85 86 87 88 90 91 92 94]

Matriz de Confusão:
[[280   8]
 [ 16 268]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96       288
           1       0.97      0.94      0.96       284

    accuracy                           0.96       572
   macro avg       0.96      0.96      0.96       572
weighted avg       0.96      0.96      0.96       572



##Gradient Boosting

In [129]:
#modelo Gradient Boosting
GB_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

pred_models(GB_model, X_admissao, y_train_test, X_test_1, X_train_1, y_test, y_train, X_val_1, y_val, results_ad)


Acurácia: 0.9353146853146853
###############################################################
Validação do modelo
###############################################################

Acurácia: 0.951048951048951

Features selecionadas
[ 0  1  2  3  5  6  9 24 25 27 28 32 33 35 40 41 42 43 44 45 46 48 62 67
 70 78 79 80 81 82 83 84 85 86 87 88 90 91 92 93]

Matriz de Confusão:
[[275  13]
 [ 15 269]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       288
           1       0.95      0.95      0.95       284

    accuracy                           0.95       572
   macro avg       0.95      0.95      0.95       572
weighted avg       0.95      0.95      0.95       572



#Treinamento utilizando informações 72 horas após a admissão no hospital

In [130]:
results = []

##SVM

---



In [131]:
#modelo SVM
svm_model = SVC(kernel='linear')
pred_models(svm_model, X_train_test, y_train_test, X_test, X_train, y_test, y_train, X_val, y_val, results)

Acurácia: 0.8251748251748252
###############################################################
Validação do modelo
###############################################################

Acurácia: 0.8199300699300699

Features selecionadas
[  0   3   4   5   8   9  10  15  19  26  27  28  32  35  40  42  44  45
  46  47  48  49  50  52  55  56  58  59  60  62  63  70  82  84  86  89
  90  97  99 101]

Matriz de Confusão:
[[224  64]
 [ 39 245]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.85      0.78      0.81       288
           1       0.79      0.86      0.83       284

    accuracy                           0.82       572
   macro avg       0.82      0.82      0.82       572
weighted avg       0.82      0.82      0.82       572



##KNN

In [132]:
#modelo KNN
knn_model = KNeighborsClassifier(n_neighbors=5)

#Treinar o modelo
model.fit(X_train, y_train)

#Testar o modelo
y_pred = model.predict(X_test)

#Avaliar o desempenho
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia da variável: {accuracy}")

print('###############################################################')
#Avaliar o desempenho
print('Validação do modelo')
print('###############################################################')
#validando o modelo
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print()
print(f"Acurácia: {accuracy}")
results.append(accuracy)
print()
relatorio(y_val, y_pred)


Acurácia da variável: 0.8426573426573427
###############################################################
Validação do modelo
###############################################################

Acurácia: 0.8146853146853147

Matriz de Confusão:
[[192  96]
 [ 10 274]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.95      0.67      0.78       288
           1       0.74      0.96      0.84       284

    accuracy                           0.81       572
   macro avg       0.85      0.82      0.81       572
weighted avg       0.85      0.81      0.81       572



##Regressão Logística

In [133]:
#modelo Regressão Logística
log_model = LogisticRegression(max_iter=1000)

pred_models(log_model, X_train_test, y_train_test, X_test, X_train, y_test, y_train, X_val, y_val, results)


Acurácia: 0.8304195804195804
###############################################################
Validação do modelo
###############################################################

Acurácia: 0.8111888111888111

Features selecionadas
[  0   3   4   5   8   9  10  11  19  26  27  28  32  35  38  40  42  44
  46  48  49  58  59  60  62  66  70  74  82  84  86  89  90  91  93  95
  97  98  99 101]

Matriz de Confusão:
[[225  63]
 [ 45 239]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.83      0.78      0.81       288
           1       0.79      0.84      0.82       284

    accuracy                           0.81       572
   macro avg       0.81      0.81      0.81       572
weighted avg       0.81      0.81      0.81       572



##Random Forest

In [134]:
#modelo Random Forest
RF_model = RandomForestClassifier(n_estimators=100, random_state=42)

pred_models(RF_model, X_train_test, y_train_test, X_test, X_train, y_test, y_train, X_val, y_val, results)

Acurácia: 0.9562937062937062
###############################################################
Validação do modelo
###############################################################

Acurácia: 0.9667832167832168

Features selecionadas
[  0   1   2   3   4   5   6   8   9  14  24  28  32  33  40  41  42  43
  44  45  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93
  95  96  99 101]

Matriz de Confusão:
[[287   1]
 [ 18 266]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       288
           1       1.00      0.94      0.97       284

    accuracy                           0.97       572
   macro avg       0.97      0.97      0.97       572
weighted avg       0.97      0.97      0.97       572



##Gradient Boosting

In [135]:
#modelo Gradient Boosting
GB_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

pred_models(GB_model, X_train_test, y_train_test, X_test, X_train, y_test, y_train, X_val, y_val, results)


Acurácia: 0.965034965034965
###############################################################
Validação do modelo
###############################################################

Acurácia: 0.9667832167832168

Features selecionadas
[  0   1   3   4   5   8   9  24  25  27  28  29  32  33  40  41  42  43
  44  45  46  48  49  70  78  79  81  82  84  85  86  88  89  90  93  95
  96  97 101 102]

Matriz de Confusão:
[[281   7]
 [ 12 272]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       288
           1       0.97      0.96      0.97       284

    accuracy                           0.97       572
   macro avg       0.97      0.97      0.97       572
weighted avg       0.97      0.97      0.97       572



In [136]:
colunas = ['Classificador', 'Admissao', 'Apos_72H']
classificadores = ['SVM', 'KNN', 'RL', 'RF', 'GB']
df_results = pd.DataFrame()

In [137]:
df_results['Classificador'] = classificadores
df_results['Admissao'] = results_ad
df_results['Apos_72H'] = results

In [138]:
df_results

,Classificador,Admissao,Apos_72H
0,SVM,0.809441,0.819930
1,KNN,0.798951,0.814685
2,RL,0.800699,0.811189
3,RF,0.958042,0.966783
4,GB,0.951049,0.966783
